In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-ipo-dataset-2010-2021/NIFTY_2010_2021.csv
/kaggle/input/indian-ipo-dataset-2010-2021/ipo_2010_2021.csv


In [2]:
# reading data
de=pd.read_csv('/kaggle/input/indian-ipo-dataset-2010-2021/ipo_2010_2021.csv')
de.sample(5)

,Date,IPO Name,Profile,Issue Size (in crores),QIB,HNI,RII,Total,Issue,Listing Open,Listing Close,Listing Gains(%),CMP,Current Gains (%)
41,30-12-19,Prince Pipes,https://www.moneycontrol.com/ipo/prince-pipes_...,500.00,3.54,1.21,1.89,2.21,178.0,160.0,166.60,-6.40,685.70,285.22
133,16-11-15,S H Kelkar,https://www.moneycontrol.com/ipo/s-h-kelkar_KC...,200.00,0.00,0.00,0.00,0.00,180.0,216.0,207.30,15.17,169.10,-6.06
101,05-04-17,Shankara Buildi,https://www.moneycontrol.com/ipo/shankara-buil...,232.96,51.62,90.68,15.35,41.88,460.0,545.0,632.80,37.57,654.25,42.23
210,04-11-10,Coal India,https://www.moneycontrol.com/ipo/coal-india_CI...,15475.00,24.70,25.40,2.31,15.28,245.0,300.0,342.35,39.73,143.25,-41.53
216,14-10-10,Sea TV Network,https://www.moneycontrol.com/ipo/sea-tv-networ...,50.20,1.66,40.94,7.45,9.58,100.0,120.0,106.00,6.00,1.98,-98.02


In [3]:
# performing the eda part

de['r']=de['Listing Close']-de['Listing Open']
de['r']=de['r'].astype(int)

# SO WE USE THE APPLY FUNCTION IN THE COLUMN

def fun(x):
    if(x>80):
        return 1
    elif(x>30 & x<80):
        return 2
    else:
        return 3
    
de['target']=de['r'].apply(fun)
de['target']
    
# target defines the risk in that ipo 1 means less risk 
#  as 3 means high risk

0      1
1      3
2      3
3      2
4      3
      ..
259    2
260    3
261    3
262    3
263    3
Name: target, Length: 264, dtype: int64

In [4]:
# FINDING THE IN SITES FROM THE DATA 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
de.isna().sum()

Date                      0
IPO Name                  0
Profile                   0
Issue Size (in crores)    0
QIB                       0
HNI                       0
RII                       0
Total                     0
Issue                     0
Listing Open              0
Listing Close             0
Listing Gains(%)          0
CMP                       0
Current  Gains (%)        2
r                         0
target                    0
dtype: int64

In [5]:
from pandas_profiling import ProfileReport
gh=ProfileReport(de)
gh.to_file(output_file='out.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
sns.kdeplot(de['RII'])

<AxesSubplot:xlabel='RII', ylabel='Density'>

In [7]:
sns.scatterplot('QIB','HNI',data=de,hue='target')

/opt/conda/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


<AxesSubplot:xlabel='RII', ylabel='Density'>

In [8]:
sns.scatterplot(y='Issue Size (in crores)',x='Listing Open',data=de)

<AxesSubplot:xlabel='RII', ylabel='Density'>

In [9]:
sns.pairplot(data=de,hue='target')

In [10]:
import plotly.express as px

fig = px.scatter_3d(de, x='Listing Open', y='Listing Close', z='Issue Size (in crores)',
              color='target')
fig.show()

In [11]:
jk=px.scatter_3d(de,x='QIB',y='HNI',z='RII',color='target')
jk.show()

In [12]:
jk=px.scatter_3d(de,x='CMP',y='Issue',z='Total',color='target')
jk.show()

In [13]:
# preparing data for the model
de.drop(columns=['Profile'],inplace=True)
de.drop(columns=['IPO Name'],inplace=True)
de

,Date,Issue Size (in crores),QIB,HNI,RII,Total,Issue,Listing Open,Listing Close,Listing Gains(%),CMP,Current Gains (%),r,target
0,29-07-21,500.00,2.55,9.78,13.36,9.50,1083.0,2111.80,2310.25,113.32,"2,268.50",109.46,198,1
1,23-07-21,9375.00,51.79,32.96,7.45,38.25,76.0,115.00,125.85,65.59,133.35,75.46,10,3
2,19-07-21,1546.00,156.37,206.43,9.00,93.41,900.0,1784.40,1585.20,76.13,"1,682.80",86.98,-199,3
3,19-07-21,963.28,93.18,73.01,4.89,43.48,837.0,1700.00,1746.80,108.70,"1,730.85",106.79,46,2
4,07-07-21,800.00,2.64,1.36,4.40,3.25,296.0,360.00,335.45,13.33,337.40,13.99,-24,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,19-02-10,0.00,0.52,1.52,2.26,1.12,75.0,70.00,71.10,-5.20,870.00,1060.00,1,2
260,15-02-10,199.80,1.12,3.65,0.62,1.22,165.0,155.90,146.38,-11.28,21.70,-86.85,-9,3
261,15-02-10,56.25,0.99,16.60,6.25,5.17,75.0,88.00,87.85,17.13,4.06,-94.59,0,3
262,08-02-10,328.70,59.39,51.95,3.79,31.11,145.0,160.00,114.50,-21.03,"3,770.20",2500.14,-45,3


In [14]:
de['Date']=pd.to_datetime(de['Date'])
print(de.dtypes)
de.isna().sum()

Date                      datetime64[ns]
Issue Size (in crores)           float64
QIB                              float64
HNI                              float64
RII                              float64
Total                            float64
Issue                            float64
Listing Open                     float64
Listing Close                    float64
Listing Gains(%)                 float64
CMP                               object
Current  Gains (%)               float64
r                                  int64
target                             int64
dtype: object


Date                      0
Issue Size (in crores)    0
QIB                       0
HNI                       0
RII                       0
Total                     0
Issue                     0
Listing Open              0
Listing Close             0
Listing Gains(%)          0
CMP                       0
Current  Gains (%)        2
r                         0
target                    0
dtype: int64

In [15]:
de.dropna(inplace=True)
de.set_index('Date',inplace=True)

In [16]:
de.drop(columns=['r'],inplace=True)
de.drop_duplicates(inplace=True)
de.reset_index(inplace=True,drop=True) # as it not the time series data so dates are not so important

In [17]:
de.sample()


,Issue Size (in crores),QIB,HNI,RII,Total,Issue,Listing Open,Listing Close,Listing Gains(%),CMP,Current Gains (%),target
122,479.21,73.18,225.3,8.73,73.55,446.0,662.0,618.1,38.59,"1,306.50",192.94,3


In [18]:
print(de.columns)
# hence due to the comma we have the type is the object so we remove the comma by the fnunction change
de['CMP']

Index(['Issue Size (in crores)', 'QIB', 'HNI', 'RII', 'Total', 'Issue',
       'Listing Open', 'Listing Close', 'Listing Gains(%)', 'CMP',
       'Current  Gains (%)', 'target'],
      dtype='object')


0      2,268.50
1        133.35
2      1,682.80
3      1,730.85
4        337.40
         ...   
257      870.00
258       21.70
259        4.06
260    3,770.20
261      473.75
Name: CMP, Length: 262, dtype: object

In [19]:
def change(x):
    f=[]
    f=x.split(',')
    t=''.join(f)
    f.clear()
    return t

de["CMP"]=de["CMP"].apply(change)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [21]:
x=de.iloc[:,:-1]
y=de.iloc[:,-1]
model=RandomForestClassifier()
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
parameterR={'max_depth':[3,2,4,5,6,7],
           'min_samples_split':[3,4,5,6],
           'n_estimators':[20,40,60,70,80],
           }
def best(model,parameter,datax,datay):
    best=GridSearchCV(model,parameter,cv=10,scoring='f1_macro',n_jobs=-1)
    best.fit(datax,datay)
    return best.best_estimator_



In [22]:
mk=best(model,parameterR,x_train,y_train)
from sklearn.metrics import accuracy_score
pred=mk.predict(x_train)
accuracy_score(y_train,pred)

0.8899521531100478

In [23]:
pred=mk.predict(x_test)
accuracy_score(y_test,pred)

0.6415094339622641